In [1]:
import wandb
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models.convnext import LayerNorm2d

import torchvision
from torchvision import datasets, models
from torchvision import transforms
from torchvision.transforms import (Compose, RandomResizedCrop, 
                                    RandomHorizontalFlip, ColorJitter, 
                                    AutoAugment, InterpolationMode, v2, ToTensor)
from torchvision.io import read_image
from torchvision.models.convnext import ConvNeXt_Tiny_Weights

import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import numpy as np
import cv2
import random
import PIL
import seaborn as sn

from torch_ema import ExponentialMovingAverage
from tqdm.auto import tqdm
import time 

import pytorch_lightning as pl

from torchmetrics import Accuracy

In [2]:
# Hyperparameters
shape = (224, 224)
epochs=50
num_classes=15
batch_size=8
learning_rate=5e-5
weight_decay = 1e-8
optimizer_momentum = (0.9, 0.999)
scale_factor = 0.8 
drop_path_rate = 0.1
label_smoothing = 0.1
num_workers = 4

# Model

In [3]:
class CustomConvNeXtTiny(pl.LightningModule):
    def __init__(self, epochs = epochs, learning_rate = learning_rate, scale_factor = scale_factor, drop_path_rate = drop_path_rate, num_classes=num_classes, label_smoothing = label_smoothing):
        super(CustomConvNeXtTiny, self).__init__()
        
        # Carregar o modelo ConvNeXt-Tiny com pesos pré-treinados
        self.model = models.convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT, drop_path_rate=drop_path_rate)
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.scale_factor = scale_factor
        self.drop_path_rate = drop_path_rate
        self.label_smoothing = label_smoothing
        
        # Definir as camadas sequenciais para classificação
        self.sequential_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LayerNorm(768, eps=1e-6, elementwise_affine=True),
            nn.Linear(in_features=768, out_features=num_classes, bias=True)
        )
        
        # Substituir a camada classifier do modelo
        self.model.classifier = self.sequential_layers
        
        # Métricas
        self.train_accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.CrossEntropyLoss()(logits, labels)

        # Calcular a precisão
        self.train_accuracy(logits, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_accuracy', self.train_accuracy, on_step=True, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.CrossEntropyLoss()(logits, labels)

        # Calcular a precisão
        self.val_accuracy(logits, labels)
        self.log('val_loss', loss, on_step=True, on_epoch=True)
        self.log('val_accuracy', self.val_accuracy, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)

        # Calcular a precisão
        self.test_accuracy(logits, labels)
        self.log('test_accuracy', self.test_accuracy, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        # Prepare the list for optimizer param groups
        param_groups = []

        # Scale learning rate for each block in the model's features
        for i, block in enumerate(self.model.features):
            lr = self.learning_rate * (self.scale_factor ** i)
            param_groups.append({'params': block.parameters(), 'lr': lr})

        # Add classifier parameters with final scaled learning rate
        classifier_lr = self.learning_rate * (self.scale_factor ** len(self.model.features))
        param_groups.append({'params': self.model.classifier.parameters(), 'lr': classifier_lr})

        # Define the optimizer with parameter groups
        optimizer = torch.optim.Adam(param_groups)

        # Define scheduler
        scheduler = CosineAnnealingLR(optimizer, T_max=self.epochs)  # Example with T_max=10

        # Return both optimizer and scheduler
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'epoch',  # Step the scheduler per epoch
                'monitor': 'val_loss',  # Optional, monitor val_loss (useful for other schedulers)
                'frequency': 1,  # Apply the scheduler every epoch
            }
        }


# Custom Image

In [4]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
from PIL import Image


class CustomImageModule(pl.LightningDataModule):
    def __init__(self, train_dir, test_dir,batch_size, num_workers):
        super().__init__()
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.image_transform = v2.Compose([
            v2.ToImage(),
            v2.Resize((224, 224), interpolation=PIL.Image.BILINEAR, antialias=False),
            v2.ToDtype(torch.uint8, scale=True),

            v2.RandomHorizontalFlip(),
            v2.RandomVerticalFlip(p=0.1),
            v2.RandomErasing(p=0.25),
            v2.RandAugment(num_ops=9, magnitude=5),

            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def setup(self, stage=None):
        """Setup training, validation, and test datasets."""
        if stage == "fit" or stage is None:
            # Load entire dataset with transforms applied
            entire_dataset = datasets.ImageFolder(root=self.train_dir, transform=self.image_transform)

            # Split dataset into training and validation (80-20 split)
            train_size = int(0.8 * len(entire_dataset))
            val_size = len(entire_dataset) - train_size
            self.train_ds, self.val_ds = random_split(entire_dataset, [train_size, val_size])

        if stage == "test" or stage is None:
            self.test_ds = datasets.ImageFolder(root=self.test_dir, transform=self.image_transform)

    def train_dataloader(self):
        """Return the training data loader."""
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )
    
    def val_dataloader(self):
        """Return the validation data loader."""
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )
    
    def test_dataloader(self):
        """Return the test data loader."""
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

# Train

In [5]:
train_dir = "imagens/swedish/train"
test_dir = "imagens/swedish/test"

In [6]:
data_module = CustomImageModule(train_dir, test_dir, batch_size, num_workers)
model = CustomConvNeXtTiny()

In [7]:
data_module

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.models.convnext as ConvNeXt
from torchvision.models import ConvNeXt_Tiny_Weights
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
from torchmetrics import Accuracy
from pytorch_lightning import Trainer

In [9]:
# Crie o Trainer e execute o treinamento
trainer = Trainer(accelerator="gpu", devices=1, precision='16-mixed', max_epochs=10)  # Ajuste o número de GPUs conforme necessário
trainer.fit(model, data_module)
trainer.test(model, data_module)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/rodrigo/miniconda3/envs/mestrado/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | model             | ConvNeXt           | 27.8 M | train
1 | sequential_layers | Sequential         | 13.1 K | train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_epoch      0.9777777791023254
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy_epoch': 0.9777777791023254}]

In [10]:
data_module.setup('test')